# Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from tqdm import tqdm
import xarray

In [3]:
from backend import loading_utils
from backend import data_paths
from backend import gauge_groups_utils
from backend import metrics_utils
from backend import evaluation_utils

In [4]:
RESTART = True

# Load Data

## Experiments

In [5]:
GOOGLE_EXPERIMENTS = ['hydrologically_separated']
GLOFAS_EXPERIMENTS = [metrics_utils.GLOFAS_VARIABLE]

### GloFAS Data

In [6]:
all_gauges = gauge_groups_utils.get_full_gauge_group()
print(f'There are {len(all_gauges)} gauges.')

There are 5678 gauges.


In [7]:
glofas_model_runs = loading_utils.load_glofas_model_runs(gauges=all_gauges)

In [8]:
glofas_gauges = set(glofas_model_runs.gauge_id.values)

### Google Data

In [9]:
google_model_runs = loading_utils.load_all_experimental_model_runs(
    gauges=glofas_gauges,
    experiments=GOOGLE_EXPERIMENTS
)

Working on experiment: hydrologically_separated


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4089/4089 [01:54<00:00, 35.67it/s]


In [10]:
google_gauges = set(google_model_runs[GOOGLE_EXPERIMENTS[0]].gauge_id.values)

### GRDC Data

In [11]:
grdc_observation_data = loading_utils.load_grdc_data()

## Overlapping Gauge Groups

In [12]:
gauges = list(glofas_gauges.intersection(google_gauges))
print(f'There are {len(gauges)} gauges that exist for both models.')

There are 4084 gauges that exist for both models.


# Time Periods

In [13]:
google_validation_time_periods = {
    gauge: ['2014-01-01', '2023-01-01'] for gauge in gauges
}

## Record Lengths

In [14]:
record_lengths = {}
for gauge in tqdm(grdc_observation_data.gauge_id):
    gauge_xr = grdc_observation_data.sel(
        {
            'gauge_id': gauge,
            'lead_time': 0,
        }
    )
    vals = gauge_xr[metrics_utils.OBS_VARIABLE].values
    vals = vals[~np.isnan(vals)]
    vals = vals[~np.isinf(vals)]
    record_lengths[str(gauge.values)] = vals.shape[0]

print(f'There are {np.sum([val for val in record_lengths.values()]) / 365} total years of GRDC observation data '
      f'from {len(record_lengths)} gauges.')

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5670/5670 [00:07<00:00, 734.29it/s]

There are 152259.27397260274 total years of GRDC observation data from 5670 gauges.


# Google Model Metrics

In [20]:
# # This needs to be uncommented for all experiments except hydrologically-separated.
# # The reason is because that experiment was run after some updates to the model that
# # included writing out the observation data to the qgis output files.

# # Add observation data to model run xarrays, 
# # and delete redundant varaible to save memory.
# for experiment in google_model_runs.keys():
#     google_model_runs[experiment] = xarray.merge(
#         [google_model_runs[experiment], grdc_observation_data])

## Metrics: 2014 - Present

In [21]:
metrics = metrics_utils.calculate_and_save_metrics_for_many_gagues_and_many_models(
    restart=RESTART,
    experiments=GOOGLE_EXPERIMENTS,
    ds=google_model_runs,
    gauges=gauges,
    sim_variable=metrics_utils.GOOGLE_VARIABLE,
    obs_variable=metrics_utils.OBS_VARIABLE,
    base_path=data_paths.PER_METRIC_GOOGLE_2014_HYDROGRAPH_METRICS_DIR,
    breakpoints_path=data_paths.PER_GAUGE_GOOGLE_2014_HYDROGRAPH_METRICS_DIR,
    time_periods=google_validation_time_periods,
)
metrics[GOOGLE_EXPERIMENTS[0]]['NSE'].replace(-np.inf, np.nan).describe()

Working on experiment: hydrologically_separated.


 10%|█████████████████████▍                                                                                                                                                                                                | 408/4084 [02:02<19:40,  3.12it/s]/home/gsnearing/miniconda3/envs/global-modeling-paper/lib/python3.11/site-packages/numpy/lib/function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/gsnearing/miniconda3/envs/global-modeling-paper/lib/python3.11/site-packages/numpy/lib/function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/home/gsnearing/miniconda3/envs/global-modeling-paper/lib/python3.11/site-packages/numpy/lib/function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/gsnearing/miniconda3/envs/global-modeling-paper/lib/python3.11/site-packages/numpy/lib/function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /

/home/gsnearing/github_repo/global_streamflow_model_paper/notebooks/backend/metrics.py:742: RuntimeWarning: divide by zero encountered in scalar divide
  fhv = np.sum(sim - obs) / np.sum(obs)
/home/gsnearing/miniconda3/envs/global-modeling-paper/lib/python3.11/site-packages/numpy/lib/function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/gsnearing/miniconda3/envs/global-modeling-paper/lib/python3.11/site-packages/numpy/lib/function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/home/gsnearing/miniconda3/envs/global-modeling-paper/lib/python3.11/site-packages/numpy/lib/function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/gsnearing/miniconda3/envs/global-modeling-paper/lib/python3.11/site-packages/numpy/lib/function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/home/gsnearing/miniconda3/envs/global-model

/home/gsnearing/github_repo/global_streamflow_model_paper/notebooks/backend/metrics.py:742: RuntimeWarning: divide by zero encountered in scalar divide
  fhv = np.sum(sim - obs) / np.sum(obs)
/home/gsnearing/miniconda3/envs/global-modeling-paper/lib/python3.11/site-packages/numpy/lib/function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/gsnearing/miniconda3/envs/global-modeling-paper/lib/python3.11/site-packages/numpy/lib/function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/home/gsnearing/miniconda3/envs/global-modeling-paper/lib/python3.11/site-packages/numpy/lib/function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/gsnearing/miniconda3/envs/global-modeling-paper/lib/python3.11/site-packages/numpy/lib/function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/home/gsnearing/miniconda3/envs/global-model

/home/gsnearing/github_repo/global_streamflow_model_paper/notebooks/backend/metrics.py:742: RuntimeWarning: divide by zero encountered in scalar divide
  fhv = np.sum(sim - obs) / np.sum(obs)
/home/gsnearing/miniconda3/envs/global-modeling-paper/lib/python3.11/site-packages/numpy/lib/function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/gsnearing/miniconda3/envs/global-modeling-paper/lib/python3.11/site-packages/numpy/lib/function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/home/gsnearing/miniconda3/envs/global-modeling-paper/lib/python3.11/site-packages/numpy/lib/function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/gsnearing/miniconda3/envs/global-modeling-paper/lib/python3.11/site-packages/numpy/lib/function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/home/gsnearing/miniconda3/envs/global-model

...

/home/gsnearing/github_repo/global_streamflow_model_paper/notebooks/backend/metrics.py:742: RuntimeWarning: divide by zero encountered in scalar divide
  fhv = np.sum(sim - obs) / np.sum(obs)
/home/gsnearing/miniconda3/envs/global-modeling-paper/lib/python3.11/site-packages/numpy/lib/function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/gsnearing/miniconda3/envs/global-modeling-paper/lib/python3.11/site-packages/numpy/lib/function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/home/gsnearing/miniconda3/envs/global-modeling-paper/lib/python3.11/site-packages/numpy/lib/function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/gsnearing/miniconda3/envs/global-modeling-paper/lib/python3.11/site-packages/numpy/lib/function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/home/gsnearing/miniconda3/envs/global-model

/home/gsnearing/miniconda3/envs/global-modeling-paper/lib/python3.11/site-packages/numpy/lib/function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/gsnearing/miniconda3/envs/global-modeling-paper/lib/python3.11/site-packages/numpy/lib/function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/home/gsnearing/miniconda3/envs/global-modeling-paper/lib/python3.11/site-packages/numpy/lib/function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/gsnearing/miniconda3/envs/global-modeling-paper/lib/python3.11/site-packages/numpy/lib/function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/home/gsnearing/miniconda3/envs/global-modeling-paper/lib/python3.11/site-packages/numpy/lib/function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/gsnearing/miniconda3/envs/global-modeli

/home/gsnearing/miniconda3/envs/global-modeling-paper/lib/python3.11/site-packages/numpy/lib/function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/gsnearing/miniconda3/envs/global-modeling-paper/lib/python3.11/site-packages/numpy/lib/function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/home/gsnearing/miniconda3/envs/global-modeling-paper/lib/python3.11/site-packages/numpy/lib/function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/gsnearing/miniconda3/envs/global-modeling-paper/lib/python3.11/site-packages/numpy/lib/function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/home/gsnearing/miniconda3/envs/global-modeling-paper/lib/python3.11/site-packages/numpy/lib/function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/gsnearing/miniconda3/envs/global-modeli

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4084/4084 [20:57<00:00,  3.25it/s]


NameError: name '_EXPERIMENTS' is not defined

## Metrics: 1980 - Present

In [23]:
metrics = metrics_utils.calculate_and_save_metrics_for_many_gagues_and_many_models(
    restart=RESTART,
    experiments=GOOGLE_EXPERIMENTS,
    ds=google_model_runs,
    gauges=gauges,
    sim_variable=metrics_utils.GOOGLE_VARIABLE,
    obs_variable=metrics_utils.OBS_VARIABLE,
    base_path=data_paths.PER_METRIC_GOOGLE_1980_HYDROGRAPH_METRICS_DIR,
    breakpoints_path=data_paths.PER_GAUGE_GOOGLE_1980_HYDROGRAPH_METRICS_DIR,
    lead_times=[0]
)
metrics[GOOGLE_EXPERIMENTS[0]]['NSE'].replace(-np.inf, np.nan).describe()

Working on experiment: hydrologically_separated.


 91%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊                   | 3717/4084 [04:50<00:26, 13.82it/s]/home/gsnearing/miniconda3/envs/global-modeling-paper/lib/python3.11/site-packages/numpy/lib/function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/gsnearing/miniconda3/envs/global-modeling-paper/lib/python3.11/site-packages/numpy/lib/function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/home/gsnearing/miniconda3/envs/global-modeling-paper/lib/python3.11/site-packages/numpy/lib/function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/gsnearing/miniconda3/envs/global-modeling-paper/lib/python3.11/site-packages/numpy/lib/function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /

,0,1,2,3,4,5,6,7
count,3.992000e+03,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,-4.266468e+03,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,2.691183e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,-1.700350e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,1.399653e-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,5.100162e-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,7.087795e-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,9.407634e-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Delete Variables to Clear Memory

In [ ]:
del google_model_runs

# GloFAS

In [ ]:
# Merge everything into one large xarray.
# This xarray merge takes ... forever ...
glofas_model_runs = xarray.merge(
    [glofas_model_runs, grdc_observation_data.sel(lead_time=0)])

## Metrics: 2014 - Present

In [ ]:
google_validation_time_periods = {
    gauge: ['2014-01-01', '2023-01-01'] for gauge in gauges
}

In [ ]:
metrics = metrics_utils.calculate_and_save_metrics_for_many_gagues_and_many_models(
    restart=RESTART,
    experiments=GLOFAS_EXPERIMENTS,
    ds={exp: glofas_model_runs for exp in GLOFAS_EXPERIMENTS},
    gauges=gauges,
    sim_variable=metrics_utils.GLOFAS_VARIABLE,
    obs_variable=metrics_utils.UNNORMALIZED_OBS_VARIABLE,
    base_path=data_paths.PER_METRIC_GLOFAS_2014_HYDROGRAPH_METRICS_DIR,
    breakpoints_path=data_paths.PER_GAUGE_GLOFAS_2014_HYDROGRAPH_METRICS_DIR,
    time_periods=google_validation_time_periods,
    lead_times=[0]
)
metrics[GLOFAS_EXPERIMENTS[0]]['NSE'].describe()

## Metrics: 1980 - Present

In [ ]:
metrics = metrics_utils.calculate_and_save_metrics_for_many_gagues_and_many_models(
    restart=RESTART,
    experiments=GLOFAS_EXPERIMENTS,
    ds={exp: glofas_model_runs for exp in GLOFAS_EXPERIMENTS},
    gauges=gauges,
    sim_variable=metrics_utils.GLOFAS_VARIABLE,
    obs_variable=metrics_utils.UNNORMALIZED_OBS_VARIABLE,
    base_path=data_paths.PER_METRIC_GLOFAS_1980_HYDROGRAPH_METRICS_DIR,
    breakpoints_path=data_paths.PER_GAUGE_GLOFAS_1980_HYDROGRAPH_METRICS_DIR,
    lead_times=[0]
)
metrics[GLOFAS_EXPERIMENTS[0]]['NSE'].describe()

# Plot Hydrograph Metric CDFs

In [ ]:
evaluation_utils.hydrograph_metrics_cdf_plots(
    glofas_basepath=data_paths.PER_METRIC_GLOFAS_2014_HYDROGRAPH_METRICS_DIR,
    google_basepath=data_paths.PER_METRIC_GOOGLE_2014_HYDROGRAPH_METRICS_DIR,
    title='2014-2022 with Lead Times',
    lead_times=list(range(8))
)

In [ ]:
evaluation_utils.hydrograph_metrics_cdf_plots(
    glofas_basepath=data_paths.PER_METRIC_GLOFAS_1980_HYDROGRAPH_METRICS_DIR,
    google_basepath=data_paths.PER_METRIC_GOOGLE_1980_HYDROGRAPH_METRICS_DIR,
    title='1984-2022 Ungauged Cross Validation Splits',
    lead_times=[0]
)